In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import re
import random 

In [ ]:
def preprocess(dataPath, stopwordsPath):
  
  with open(stopwordsPath) as f:
    stopwords = [line.strip() for line in f]

  with open(dataPath) as f:
    docs = [doc.strip() for doc in f]
  
  N = len(docs)

  word2id = {}
  id2word = {}
  currId = 0
  wordids = list()
  wordcts = list()

  for doc in docs:  
    doc = doc.lower()
    doc = re.sub(r'-', ' ', doc)
    doc = re.sub(r'[^a-z]', ' ', doc)
    doc = re.sub(r' +', ' ', doc)
    words = doc.split()
    wordCount = {}    
    
    for word in words:
      word = word.lower().strip()
      if len(word)>1 and word not in stopwords and not re.search('[0-9]', word):
        if word not in word2id.keys():
          word2id[word] = currId
          id2word[currId] = word
          currId += 1
        if word in wordCount:
          wordCount[word] += 1
        else: wordCount[word] = 1

    ids = []
    for word in wordCount.keys():
      ids.append(word2id[word])
    wordids.append(ids)
    wordcts.append(list(wordCount.values()))

  M = len(word2id)

  return N, M, (wordids, wordcts), word2id, id2word


In [ ]:
def init(K):
  theta = np.random.rand(N, K)
  beta = np.random.rand(K,M)
  for n in range(0, N):
    normalization = sum(theta[n, :])
    theta[n, :] = theta[n, :] / normalization
  for k in range(0, K):
    normalization = sum(beta[k, :])
    beta[k, :] = beta[k,:] / normalization
  return theta, beta

In [ ]:
theta, beta = init(K)

ids = wordids[0]
cts = wordcts[0]

thetad = theta[0, :]
betad = beta[:, ids]

print("shape thetad:", thetad.shape)
print("shape betad:", betad.shape)
print("shape of sum:", np.sum(betad, axis=1).shape)
a = (thetad * np.sum(betad, axis=1)) / sum(cts)
print(a)

shape thetad: (10,)
shape betad: (10, 171)
shape of sum: (10,)
[8.89971106e-07 1.76284264e-05 1.79170880e-05 1.11236559e-06
 1.72732532e-05 9.62232781e-06 1.42350131e-05 1.08435072e-05
 1.60445396e-05 7.39686682e-06]


In [ ]:
theta, beta = init(K)

In [ ]:
def logLikelihood():
  res = 0
  for d in range(0, N):
    ids = wordids[d]
    thetad = theta[d,:]
    betad = beta[:, ids]
    res += np.sum(np.dot(thetad, betad))
  return res

def EM(max_iter, threshold):
  old_loglikelihood = logLikelihood()

  for iter in range(0, max_iter):
    beta_component = np.zeros((K, M))
    for d in range(0, N):
      ids = wordids[d]
      cts = wordcts[d]

      thetad = theta[d, :]
      betad = beta[:, ids]

      to_norm = np.dot(thetad, betad)

      theta[d,:] = (thetad * np.sum(betad / to_norm, axis=1)) / sum(cts)

      beta_component[:, ids] += np.expand_dims(thetad,axis=1) * ((betad / to_norm) * cts)

    beta = beta_component / np.sum(beta_component, axis=0)

    new_loglikelihood = logLikelihood()
    print("old:", old_loglikelihood)
    print("new:", new_loglikelihood)
    if (new_loglikelihood - old_loglikelihood < threshold):
      break
    old_loglikelihood = new_loglikelihood



In [ ]:
def output():
  for i in range(0, K):
    print("Topic", i)
    topicword = []
    ids = (beta[i, :]).argsort()
    for j in ids:
      topicword.append(id2word[j])
    for word in range(0, min(10, len(topicword))):
      print(topicword[word], end=" ")
    print('\n')

In [66]:
class PLSI:
  
  def __init__(self, max_iter, tol, dataPath, stopwordsPath, K):
    self._K = K
    self._max_iter = max_iter
    self._tol = tol
    self._dataPath = dataPath
    self._stopwordsPath = stopwordsPath
    
  def preprocess(self):
    
    with open(self._stopwordsPath) as f:
      stopwords = [line.strip() for line in f]

    with open(self._dataPath) as f:
      docs = [doc.strip() for doc in f]
    
    self._N = len(docs)

    self._word2id = {}
    self._id2word = {}
    currId = 0
    self._wordids = list()
    self._wordcts = list()

    for doc in docs:  
      doc = doc.lower()
      doc = re.sub(r'-', ' ', doc)
      doc = re.sub(r'[^a-z]', ' ', doc)
      doc = re.sub(r' +', ' ', doc)
      words = doc.split()
      wordCount = {}    
      
      for word in words:
        word = word.lower().strip()
        if len(word)>1 and word not in stopwords and not re.search('[0-9]', word):
          if word not in self._word2id.keys():
            self._word2id[word] = currId
            self._id2word[currId] = word
            currId += 1
          if word in wordCount:
            wordCount[word] += 1
          else: wordCount[word] = 1

      ids = []
      for word in wordCount.keys():
        ids.append(self._word2id[word])
      self._wordids.append(ids)
      self._wordcts.append(list(wordCount.values()))

    self._M = len(self._word2id)

  def init(self):
    np.random.seed(0)
    self._theta = np.random.rand(self._N, self._K)
    self._beta = np.random.rand(self._K,self._M)
    for n in range(0, self._N):
      normalization = sum(self._theta[n, :])
      self._theta[n, :] = self._theta[n, :] / normalization
    for k in range(0, self._K):
      normalization = sum(self._beta[k, :])
      self._beta[k, :] = self._beta[k,:] / normalization

  def logLikelihood(self):
    res = 0
    for d in range(0, self._N):
      ids = self._wordids[d]
      cts = self._wordcts[d]
      thetad = self._theta[d,:]
      for j in ids:
        res += np.log(np.dot(thetad, self._beta[:, j])) * cts[ids.index(j)]
    return res

  def EM(self):
    old_loglikelihood = self.logLikelihood()

    for iter in range(0, self._max_iter):
      beta_component = np.zeros((self._K, self._M))
      for d in range(0, self._N):
        ids = self._wordids[d]
        cts = self._wordcts[d]

        thetad = self._theta[d, :]
        betad = self._beta[:, ids]

        to_norm = np.dot(thetad, betad)

        self._theta[d,:] = (thetad * np.sum(betad / to_norm * cts, axis=1)) / sum(cts)

        beta_component[:, ids] += np.expand_dims(thetad,axis=1) * ((betad / to_norm) * cts)

      self._beta = beta_component / np.expand_dims(np.sum(beta_component, axis=1), axis=1)

      new_loglikelihood = self.logLikelihood()
      print("iter:", iter)
      print("old:", old_loglikelihood)
      print("new:", new_loglikelihood)
      if (new_loglikelihood - old_loglikelihood < self._tol):
        break
      old_loglikelihood = new_loglikelihood

  def output(self):
    for i in range(0, self._K):
      print("Topic", i)
      topicword = []
      ids = (self._beta[i, :]).argsort()
      for j in ids:
        topicword.insert(0, self._id2word[j])
      for word in range(0, min(10, len(topicword))):
        print(topicword[word], end=" ")
      print('\n')

  def EM2(self):
    old_loglikelihood = self.logLikelihood()
    to = np.zeros((self._N, self._M, self._K))
    for iter in range(self._max_iter):
      # e_step
      for d in range(0,self._N):
        ids = self._wordids[d]
        for n in ids:
          norm_k = 0
          for k in range(0, self._K):
            to[d,n,k] = self._theta[d,k] * self._beta[k,n] 
            norm_k += to[d,n,k]
          for k in range(0, self._K):
            to[d,n,k] /= norm_k
    
      # m_step
      # update theta
      for d in range(0, self._N):
        ids = self._wordids[d]
        cts = self._wordcts[d]
        for k in range(0, self._K):
          self._theta[d,k] = 0
          for n in ids:
            self._theta[d,k] += to[d,n,k] * cts[ids.index(n)]
          self._theta[d,k] /= sum(cts)
      # update beta
      for k in range(0, self._K):
        norm_j = 0
        for j in range(0, self._M):
          self._beta[k,j] = 0
          for d in range(0, self._N):
            ids = self._wordids[d]
            cts = self._wordcts[d]
            if j in ids:
              idx = ids.index(j)
              self._beta[k,j] += cts[idx] * to[d,j,k]
          norm_j += self._beta[k,j]
        for j in range(0, self._M):
          self._beta[k,j] /= norm_j

      new_loglikelihood = self.logLikelihood()
      print("iter:", iter)
      print("old:", old_loglikelihood)
      print("new:", new_loglikelihood)
      if (new_loglikelihood - old_loglikelihood < self._tol):
        break
      old_loglikelihood = new_loglikelihood
    
    self._to = to

In [67]:
dataPath = "/content/drive/My Drive/Colab Notebooks/LAB/Some Models/PLSI/dataset2.txt"
stopwordsPath = "/content/drive/My Drive/Colab Notebooks/LAB/Some Models/PLSI/stopwords.dic"

K = 10
max_iter = 20
tol = 10.0

plsi = PLSI(max_iter, tol, dataPath, stopwordsPath, K)


In [68]:
plsi.preprocess()


In [69]:
plsi.init()
plsi.EM2()
plsi.output()

iter: 0
old: -161129.26056834045
new: -147490.1087418391
iter: 1
old: -147490.1087418391
new: -145094.7533286213
iter: 2
old: -145094.7533286213
new: -141859.22296432525
iter: 3
old: -141859.22296432525
new: -138166.5713788146
iter: 4
old: -138166.5713788146
new: -134702.02753051175
iter: 5
old: -134702.02753051175
new: -131928.68138301602
iter: 6
old: -131928.68138301602
new: -129885.72768146568
iter: 7
old: -129885.72768146568
new: -128382.23918464764
iter: 8
old: -128382.23918464764
new: -127248.53571419256
iter: 9
old: -127248.53571419256
new: -126385.07388998519
iter: 10
old: -126385.07388998519
new: -125716.9750676066
iter: 11
old: -125716.9750676066
new: -125203.0741985239
iter: 12
old: -125203.0741985239
new: -124827.77063270802
iter: 13
old: -124827.77063270802
new: -124547.2589346319
iter: 14
old: -124547.2589346319
new: -124336.74772982775
iter: 15
old: -124336.74772982775
new: -124177.70425092605
iter: 16
old: -124177.70425092605
new: -124050.75726202774
iter: 17
old: -1240

In [62]:
plsi.init()

In [63]:

plsi.EM()
plsi.output()

iter: 0
old: -161129.26056834045
new: -147521.97155748782
iter: 1
old: -147521.97155748782
new: -144514.56722731955
iter: 2
old: -144514.56722731955
new: -140120.9172173746
iter: 3
old: -140120.9172173746
new: -135351.9043962023
iter: 4
old: -135351.9043962023
new: -131550.7549654084
iter: 5
old: -131550.7549654084
new: -128897.79986439472
iter: 6
old: -128897.79986439472
new: -127128.14500525457
iter: 7
old: -127128.14500525457
new: -125965.9534177318
iter: 8
old: -125965.9534177318
new: -125204.80860313786
iter: 9
old: -125204.80860313786
new: -124711.7005655186
iter: 10
old: -124711.7005655186
new: -124353.42764977404
iter: 11
old: -124353.42764977404
new: -124076.04624864932
iter: 12
old: -124076.04624864932
new: -123881.81579112173
iter: 13
old: -123881.81579112173
new: -123764.09798884019
iter: 14
old: -123764.09798884019
new: -123683.74041467332
iter: 15
old: -123683.74041467332
new: -123626.19751354681
iter: 16
old: -123626.19751354681
new: -123588.40420854598
iter: 17
old: -12

In [ ]:
thetad = plsi._theta[0]    # 1*10
cts = plsi._wordcts[0]     # 1*171
ids = plsi._wordids[0]
betad = plsi._beta[:, ids] # 10*171
to_norm = np.dot(thetad, betad)  # 1*171

res = thetad * np.sum(betad/to_norm*cts, axis=1) /sum(cts)

# beta_component[:, ids] += np.expand_dims(thetad,axis=1) * ((betad / to_norm) * cts)
# self._beta = beta_component / np.sum(beta_component, axis=0)

ans = np.expand_dims(thetad,axis=1) * ((betad / to_norm) * cts)
print(ans.shape)
#print((np.sum(betad/to_norm*cts, axis=1)).shape)

(10, 171)


In [ ]:
from numpy import zeros, int8, log
from pylab import random
import sys
import jieba
import re
import time
import codecs

# segmentation, stopwords filtering and document-word matrix generating
# [return]:
# N : number of documents
# M : length of dictionary
# word2id : a map mapping terms to their corresponding ids
# id2word : a map mapping ids to terms
# X : document-word matrix, N*M, each line is the number of terms that show up in the document
def preprocessing(datasetFilePath, stopwordsFilePath):
    
    # read the stopwords file
    file = codecs.open(stopwordsFilePath, 'r', 'utf-8')
    stopwords = [line.strip() for line in file] 
    file.close()
    
    # read the documents
    file = codecs.open(datasetFilePath, 'r', 'utf-8')
    documents = [document.strip() for document in file] 
    file.close()

    # number of documents
    N = len(documents)

    wordCounts = [];
    word2id = {}
    id2word = {}
    currentId = 0;
    # generate the word2id and id2word maps and count the number of times of words showing up in documents
    for document in documents:
        segList = jieba.cut(document)
        wordCount = {}
        for word in segList:
            word = word.lower().strip()
            if len(word) > 1 and not re.search('[0-9]', word) and word not in stopwords:               
                if word not in word2id.keys():
                    word2id[word] = currentId;
                    id2word[currentId] = word;
                    currentId += 1;
                if word in wordCount:
                    wordCount[word] += 1
                else:
                    wordCount[word] = 1
        wordCounts.append(wordCount);
    
    # length of dictionary
    M = len(word2id)  

    # generate the document-word matrix
    X = zeros([N, M], int8)
    for word in word2id.keys():
        j = word2id[word]
        for i in range(0, N):
            if word in wordCounts[i]:
                X[i, j] = wordCounts[i][word];    

    return N, M, word2id, id2word, X

def initializeParameters():
    for i in range(0, N):
        normalization = sum(lamda[i, :])
        for j in range(0, K):
            lamda[i, j] /= normalization;

    for i in range(0, K):
        normalization = sum(theta[i, :])
        for j in range(0, M):
            theta[i, j] /= normalization;

def EStep():
    for i in range(0, N):
        for j in range(0, M):
            denominator = 0;
            for k in range(0, K):
                p[i, j, k] = theta[k, j] * lamda[i, k];
                denominator += p[i, j, k];
            if denominator == 0:
                for k in range(0, K):
                    p[i, j, k] = 0;
            else:
                for k in range(0, K):
                    p[i, j, k] /= denominator;

def MStep():
    # update theta
    for k in range(0, K):
        denominator = 0
        for j in range(0, M):
            theta[k, j] = 0
            for i in range(0, N):
                theta[k, j] += X[i, j] * p[i, j, k]
            denominator += theta[k, j]
        if denominator == 0:
            for j in range(0, M):
                theta[k, j] = 1.0 / M
        else:
            for j in range(0, M):
                theta[k, j] /= denominator
        
    # update lamda
    for i in range(0, N):
        for k in range(0, K):
            lamda[i, k] = 0
            denominator = 0
            for j in range(0, M):
                lamda[i, k] += X[i, j] * p[i, j, k]
                denominator += X[i, j];
            if denominator == 0:
                lamda[i, k] = 1.0 / K
            else:
                lamda[i, k] /= denominator

# calculate the log likelihood
def LogLikelihood():
    loglikelihood = 0
    for i in range(0, N):
        for j in range(0, M):
            tmp = 0
            for k in range(0, K):
                tmp += theta[k, j] * lamda[i, k]
            if tmp > 0:
                loglikelihood += X[i, j] * log(tmp)
    return loglikelihood

# output the params of model and top words of topics to files
def output():
  
    # top words of each topic
    for i in range(0, K):
        topicword = []
        ids = theta[i, :].argsort()
        for j in ids:
            topicword.insert(0, id2word[j])
        tmp = ''
        for word in topicword[0:min(topicWordsNum, len(topicword))]:
            tmp += word + ' '

        print("topic ", i)
        print(tmp)
    
# set the default params and read the params from cmd
datasetFilePath = 'dataset.txt'
stopwordsFilePath = 'stopwords.dic'
threshold = 10.0
datasetFilePath = "/content/drive/My Drive/Colab Notebooks/LAB/Some Models/PLSI/dataset2.txt"
stopwordsFilePath = "/content/drive/My Drive/Colab Notebooks/LAB/Some Models/PLSI/stopwords.dic"
K = 10
maxIteration = 20

# preprocessing
N, M, word2id, id2word, X = preprocessing(datasetFilePath, stopwordsFilePath)

np.random.seed(0)
# lamda[i, j] : p(zj|di)
lamda = random([N, K])

# theta[i, j] : p(wj|zi)
theta = random([K, M])

# p[i, j, k] : p(zk|di,wj)
p = zeros([N, M, K])


initializeParameters()

# EM algorithm
oldLoglikelihood = 1
newLoglikelihood = 1
for i in range(0, maxIteration):
    EStep()
    MStep()
    newLoglikelihood = LogLikelihood()
    print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
    print("theta0:", lamda[0])
    if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
        break
    oldLoglikelihood = newLoglikelihood



In [35]:
plsi._theta = lamda
plsi._beta = theta
print(plsi.logLikelihood())
print(LogLikelihood())

-229513.3658809677
-124024.8353040919


In [29]:
topicWordsNum = 10
output()

topic  0
percent bank prices government oil rate rose police month inflation 
topic  1
barry fbi percent gorbachev greyhound company union moore study program 
topic  2
administration soviet noriega percent duracell gorbachev farmer people president panama 
topic  3
official scientists magellan peres spacecraft offer grain contact receptor bechtel 
topic  4
gas record percent nelson summer tuesday monday immigration power season 
topic  5
soviet percent children nikolais immigration calgary month manufacturing drug anthrax 
topic  6
people california soviet central roberts waste polish nation snow northern 
topic  7
rating people bush campaign florio congress president exxon jewish washington 
topic  8
dukakis bush president people saturday plant saudi front american school 
topic  9
fire owned north summit friday officials reported police black kim 
